*Author:wxz*  
*2018/08/06*  
*Email:876688461@qq.com*  
*[Jupyter version in github](https://github.com/johnsonkee/recommend/blob/master/README)*

# 用mxnet搭建网络
下面使用`gluon`模块来搭建一个简单的网络
## 导入mxnet函数包

In [10]:
# import dependencies
from __future__ import print_function
import numpy as np
import mxnet as mx
import mxnet.ndarray as F
import mxnet.gluon as gluon
from mxnet.gluon import nn
from mxnet import autograd    # 自动求导模块

`gluon`里有`nn`模块和`rnn`模块，构建多层感知器以及深度神经网络可以直接使用`nn`模块，若是想构建循环神经网络得使用`rnn`模块。
## 定义网络
下面介绍两种定义网络的方式，第一种方式构建简便，但网络结构单一；第二种方式构建过程稍微复杂，但是网络结构更加灵活。

In [ ]:
# 方式一
net = nn.Sequential()
with net.name_scope():
    net.add(
        # add函数可以让网络一层一层地搭建，网络里信息流动的方向是从上到下
        # 在卷积层Conv2里，channels=6 意味着输出的维数为6，gluon模块里，
        # 允许网络忽略输入向量的维数，输入的维数会在网络接受第一个输入向量
        # 时，由输入向量的维数来指定。同时也可以通过赋值给参数`in_channels`
        # 来指定输入维度的大小。这里指定了激活函数为`relu`,如果不指定激活函
        # 数，默认使用`linear`，f(x)=x。
        nn.Conv2D(channels=6, kernel_size=5, activation='relu'),
        # 最大池化层
        nn.MaxPool2D(pool_size=(2,2)),
        nn.Conv2D(channels=16, kernel_size=5, activation='relu'),
        nn.MaxPool2D(pool_size=(2,2)),
        # flatten使上面网络生成的4-D矩阵变换成2-D矩阵，以便能够输入到下面
        # 的Dense层(即全连接层fully-connected layer)
        nn.Flatten(),
        nn.Dense(120),
        nn.Dense(84),
        nn.Dense(10)
    )
net

In [11]:
# 方式二
# gluon.Block是最基本的网络
class Net(gluon.Block):
    def __init__(self, **kwargs):  # 可以传入参量控制网络的参数
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            # layers created in name_scope will inherit name space
            # from parent layer.
            self.conv1 = nn.Conv2D(6, kernel_size=5)
            self.pool1 = nn.MaxPool2D(pool_size=(2,2))
            self.conv2 = nn.Conv2D(16, kernel_size=5)
            self.pool2 = nn.MaxPool2D(pool_size=(2,2))
            self.fc1 = nn.Dense(120)
            self.fc2 = nn.Dense(84)
            self.fc3 = nn.Dense(10)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        # 0 means copy over size from corresponding dimension.
        # -1 means infer size from the rest of dimensions.
        x = x.reshape((0, -1))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


## 初始化参数
在使用mxnet时，要记住，一个网络在使用前必须先初始化。（注：以下的过程建立在方式二上）

In [12]:
net = Net()
# Initialize on CPU. Replace with `mx.gpu(0)`, or 
# `[mx.gpu(0), mx.gpu(1)]`,etc to use one or more GPUs.
net.initialize(mx.init.Xavier(), ctx=mx.cpu())
# 还有一种初始化网络的方法，有时候我们需要对不同的网络层采用不一样的
# 初始化方式，我们可以在网络定义时使用`weight_initializer`这个参
# 数来指定不同的初始化方式，然后在生成对象时，直接使用`net.initialize()`
# 来进行初始化，具体可以参考下面这个例子

An example:
```
class Net(gluon.Block):
    def __init__(self, **kwargs):  # 可以传入参量控制网络的参数
        super(Net, self).__init__(**kwargs)
        with self.name_scope():
            # conv1 使用了正态分布的初始化方式
            self.conv1 = nn.Conv2D(6, kernel_size=5, weight_initializer=mx.init.Normal())
            self.pool1 = nn.MaxPool2D(pool_size=(2,2))
            # fc1 使用了Xavier的初始化方式
            self.fc1 = nn.Dense(120,weight_initializer=mx.init.Xavier())

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = x.reshape((0, -1))
        x = F.relu(self.fc1(x))
        return x
net = Net()
# 由于网络定义时已经定义了初始化参数的方式，这里就不需要指定初始化参数的方式了
net.initialize(ctx=mx.cpu())
```

## 把数据传入网络

In [13]:
data = mx.nd.random_normal(shape=(10, 1, 32, 32))  # dummy data
output = net(data)

## 损失函数
损失函数loss functions会计算网络的输出outputs和真实的标签label之间的差距。在`gluon.loss`函数包里有已经定义好的损失函数，在这里我们使用`softmax_cross_entropy_loss`来作为我们的损失函数

In [14]:
L = gluon.loss.SoftmaxCrossEntropyLoss()

## 优化方法
拥有损失函数之后，我们需要定义我们的优化方法来优化每层网络的参数，优化方法有`sgd`,`adam`等等，和优化相关的一个超参数叫做学习率(learning rates)，在定义我们的优化方法时需要确定学习率的大小，大的学习率可以加快训练的速度，但不容易收敛；小的学习率刚好相反。
在`gluon`模块里，提供了`Trainer`类来帮助我们定义我们自己的优化方法

In [15]:
lr = 0.01 # learning rate
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate':lr})

## 反向传播更新权值
`gluon`里提供了自动求导的类`autograd`，因此我们无需亲自推导网络反向传播的函数表达式，只用通过下面代码就能够完成反向传播和更新网络权值的过程

In [17]:
label = mx.nd.arange(10)  # dummy label
with autograd.record():  # 放在这句话之内，才能自动求导
    output = net(data)
    L = gluon.loss.SoftmaxCrossEntropyLoss()
    loss = L(output, label)
loss.backward()   # 反向传播

# do the update. Trainer needs to know the batch size of data to normalize
# the gradient by 1/batch_size.
trainer.step(data.shape[0]) # 括号里一般是一个batch的大小